# Using IPython Notebook:
**!!! RUN THE BOEXES BY SELECTING IT and press: CRTL+ENTER or the button ► !!!** <br>
This will cause the code in the box to be executed.


In [3]:
from IPython.display import Markdown, display
import os
import sys
print('CIAO')

CIAO


## Install istSOS on your Debian machine

**Execute in a shell the following commands to install istSOS:**

wget https://sourceforge.net/projects/istsos/files/python3-istsos_2.4.0-RC2.deb

sudo dpkg -i python3-istsos_2.4.0-RC2.deb;sudo apt-get -f -y install

sudo apt-get install postgresql-10 postgresql-10-postgis-2.4

sudo -u postgres psql -c "alter user postgres password 'postgres';"

sudo -u postgres psql -d istsos -c 'CREATE EXTENSION postgis'


## Set your server url and your initial in the variables below
In this workshop use your **initial** + **demo**. (e.g.: MCdemo)<br>
Note the 'xx' to contain strings in python<br>
**!!! DON'T USE SPACES OR SPECIAL CHARACTERS !!!**

In [4]:
#======= CONFIGURE HERE ======================
sos_server = 'http://istsos.org/istsos/'
initials= 'max'
#======== STOP CONFIGURE ======================


sos_instance = initials + "demo"
sos_url = sos_server + sos_instance

print("YOUR INSTANCE NAME IS: ", sos_instance)
print()
print("YOUR SERVICE URL IS: ", sos_url)

YOUR INSTANCE NAME IS:  maxdemo

YOUR SERVICE URL IS:  http://istsos.org/istsos/maxdemo


# istSOS introduction

## 1. Access to the istSOS online portal
Access to istSOS portal is generally protected by username and password <br>
for this tutorial, we'll use a fully open service <br>
To access the Web Administration Interface select the foowing link (SERVICE + admin)



In [5]:
display(Markdown(" " + sos_server + 'admin/'))

 http://istsos.org/istsos/admin/

You'll land to the istSOS portal where you have listed the available istSOS services: <br>
we'll use the instance named "demo" <br>

⚒ Explore the buttons <br>

## 2. Create a new service instance
Now that you have configured istSOS, it’s time to create a new service instance. <br>

---
⚒ push the **New service** icon <br>
Use the name of your "YOUR INSTANCE NAME" instance generated at veraible setup. <br>

→Here may want to change the default EPSG (and eventually the database configuration)  <br>
→For this istSOS instance you prefer to use the defaults ones. 

---

⚒  Press “next”. <br> 
As you pressed the “next” button, the server status panel is displayed.<br>
→If something's gone wrong during the creation you will see here an alert.

---

⚒  Check if the new service exists asking for a **GetCapabilities** <br>
Now the istSOS **demo** instance is up and running.<br>
→Let’s try to execute a getCapabilities request. <br>



In [6]:
display(Markdown(" " + sos_url + "?request=getCapabilities&service=SOS"))

 http://istsos.org/istsos/maxdemo?request=getCapabilities&service=SOS

## 3. Access to the istSOS online portal

⚒ Set your service provider and identification metadata <br>
⚒ Test your database conncection <br>
⚒ Check the quality indexes (optionally adjust them to your needs) <br>
⚒ Adjust your Proxy address (this is the address used internally by the application, not the public url) <br>


In [7]:
print("YOUR PROXY URL IS: ", sos_url)

YOUR PROXY URL IS:  http://istsos.org/istsos/maxdemo


## 4. Registering new sensors 
From the “services” drop down button choose **"YOUR INSTANCE NAME"** instance.

**ATTENTION:**
- Once a procedure is created the outputs (observed properties) cannot be changed. 
- Before registering new sensors it’s advised to initialize missing observed properties and unit of measures
- Is it possible to add new procedures observing more than one properties, but in istSOS this means that for each observation instant we have a list all the observed properties values.

---

⚒ Add procedures **W_MAGGIA** with the following metadata:
- **Name**: W_MAGGIA
- **Description**: Water table height monitoring, Ticino. 
- **Keywords**: hydrogeology, groundwater, 
- **System type**: insitu-fixed-point 
- **Sensor type**: Pietzometer 
- **FOI name**: MAGGIA 
- **EPSG**: 21781 
- **Coordinates**: x: 691345 y: 128105 z: 388 
- **Outputs**: 
  - **Observed property**: urn:ogc:def:parameter:x-istsos:1.0:ground:water:height
  - **Unit of measure**: m
  - **Description**: water table level in m.a.s.l

Register the new sensor (procedure) pressing the "submit" button

---

⚒  Verify the inserted procedures using the administration interface<br>
→Check your procedures by accessing the *Procedures* panel. <br>
→You will see a table showing an abstract of all the inserted procedures.<br>
→By clicking on the name you will be able to enter the details metadata that you configured during the procedure registration.<br>

**NOTE:** <br>
→The *Procedures* panel not only allows for procedures and metadata exploration but also allows details modification.<br> 
→The only exception are the outputs parameters.<br>

---

⚒ Verify the inserted procedures using the Sensor Observation Service with a getCapabilities request
 

In [ ]:
display(Markdown(
    " " + sos_url + "?request=getCapabilities&section=contents&service=SOS"))

Let’s execute a describeSensor request to verify that the procedure description is available: 

In [ ]:
display(Markdown(
    " " + sos_url + "?" +
    "request=DescribeSensor&" + 
    "procedure=W_MAGGIA&" + 
    "outputFormat=text%2Fxml%3Bsubtype%3D%22sensorML%2F1.0.1%22&" +
    "service=SOS&version=1.0.0"))

**NOTE:** Procedures are stored in the SOS with a uniqueID which is urn:ogc:def:procedure:x-istsos:1.0:XXXX for procedure named XXXX. <br> 
istSOS, as we can see later for other parameters also, is not strict and allows to specify just the procedure name in the request

## 5. Insert observations

in this part we're using a library named OAT which has been implemented within the <br>
**H2020 FREEWAT (http://freewat.eu) project**

⚒ Import OAT libraries

In [ ]:
from oatlib import sensor, oat_utils

⚒ Create an OAT-Sensor object by getting metadata from istSOS

In [ ]:
observed_property_name = "urn:ogc:def:parameter:x-istsos:1.0:ground:water:height"
observed_property_uom = "m"
basic_auth = None
procedure_name = "W_MAGGIA"

SENSOR = sensor.Sensor.from_istsos(
    sos_url, procedure_name, observed_property_name, basic_auth, srid=21781)

print(SENSOR)

⚒ Load data from a CSV to the OAT-Sensor object

In [ ]:
SENSOR.ts_from_csv(
    'W_MAGGIA_min.csv',
    sep=',',
    skiprows=1
)

⚒ Let's see the OAT-Sensor object summary information

In [ ]:
print(SENSOR)

⚒ Upload OAT-Sensor data to istSOS procedure W_MAGGIA

In [ ]:
oat_utils.sensors_to_istsos(
    service=sos_url,
    procedure=procedure_name,
    obspro_sensor={'urn:ogc:def:parameter:x-istsos:1.0:ground:water:height': SENSOR},
    how_merge='left'
)

### ⚒ Let's go and check the data in istSOS using the *DataManagement* interface and plot data of the last 2 years.